In [3]:
from cnn import ScriptClassificationCNNModel
from bdrc_script_classification_dataset import BDRCScriptClassification
from mutils import train_val_test_split

from torch.utils.data import DataLoader
import torch.nn as nn
import torch
from torchvision import transforms
from pytorch_lightning import Trainer

import mlflow

In [7]:
ds_path = '/Users/hjklvfr/Projects/ocr/data/ScriptClassification/Data'
ds_path

'/Users/hjklvfr/Projects/ocr/data/ScriptClassification/Data'

In [4]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import pytorch_lightning as L
from pytorch_lightning.loggers import MLFlowLogger
from cnn import LitModel
from cnn import MyDataModule
from datetime import datetime

from dotenv import load_dotenv
load_dotenv()
import os
print(os.environ['MLFLOW_S3_ENDPOINT_URL'])

mlflow.set_tracking_uri("https://mlflow.planetmc.ru")
mlflow.pytorch.autolog(checkpoint=True)

mlf_logger = MLFlowLogger(
    experiment_name='team_experiment',
    run_name='experiment_v2',
    tracking_uri='https://mlflow.planetmc.ru',  # Remote server
    log_model='all'
)

# Keep the top 5 best models
checkpoints_path = f'checkpoints-{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}/'
checkpoint_callback = ModelCheckpoint(
    dirpath=checkpoints_path,
    filename='model-{epoch:02d}-{val_loss:.2f}',
    monitor='val_loss',
    mode='min',
    save_top_k=5,  # Save top 5 models
    verbose=True
)


early_stop = EarlyStopping(monitor='val_accuracy', mode='max', patience=5)



# Usage with DataModule
datamodule = MyDataModule(batch_size=32)
model = LitModel(learning_rate=1e-3)

trainer = L.Trainer(max_epochs=10, accelerator='auto', devices=1, callbacks=[checkpoint_callback], logger=mlf_logger)

# Pass DataModule directly
with mlflow.start_run() as run:
    trainer.fit(model, datamodule)

trainer.test(model, datamodule, ckpt_path='best')

INFO: GPU available: True (mps), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (mps), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores


https://api.minio.planetmc.ru


2026/01/19 02:53:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/hjklvfr/Projects/workbench/.venv/lib/python3.12/site-packages/mlflow/pytorch/_lightning_autolog.py:467: UserWarning: Autologging is known to be compatible with pytorch-lightning versions between 2.0.7 and 2.5.2 and may not succeed with packages outside this range."


┏━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name  ┃ Type     ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model │ SimpleNN │  118 K │ train │     0 │
└───┴───────┴──────────┴────────┴───────┴───────┘

Trainable params: 118 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 118 K                                                                                                
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 5                                                                                           
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

...

🏃 View run stately-fowl-931 at: https://mlflow.planetmc.ru/#/experiments/0/runs/04a2660bdd164bc6b73b9c7498742a6c
🧪 View experiment at: https://mlflow.planetmc.ru/#/experiments/0


/Users/hjklvfr/Projects/workbench/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.07999999821186066    │
│         test_loss         │    2.3370609283447266     │
└───────────────────────────┴───────────────────────────┘

🏃 View run experiment_v2 at: https://mlflow.planetmc.ru/#/experiments/3/runs/468e3b9d30d0434895c0d1ac355dd1c1
🧪 View experiment at: https://mlflow.planetmc.ru/#/experiments/3


[{'test_loss': 2.3370609283447266, 'test_accuracy': 0.07999999821186066}]

In [ ]:


# transform = transforms.ToTensor()

# transform = transforms.Compose([
#     transforms.Resize(size=(600,600)),
#     transforms.ToTensor(),
#     transforms.Normalize((.4005), (.7993))
# ])


# device = torch.device('mps')
# lr = 1e-3

# models = {
#     "script_classification_cnn": {"device": device, "model": ScriptClassificationCNNModel, "criterion": nn.CrossEntropyLoss(), "lr": 1e-3, "batch_size": 1, "num_epochs": 1, "transform": transform},
# }

# for model_name, config in models.items():
#     print(model_name)
#     model = config["model"]().to(config["device"])
#     criterion = config["criterion"]
#     optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])
#     batch_size = config["batch_size"]
#     epochs = config["num_epochs"]
#     transform = config["transform"]
    

#     dataset_overall = BDRCScriptClassification(ds_path, transform)
#     train_ds, val_ds, test_ds = train_val_test_split(dataset_overall)

#     train_loader = DataLoader(train_ds, batch_size, shuffle=True)
#     test_loader = DataLoader(test_ds, batch_size, shuffle=False)


#     for epoch in range(epochs):
#         print(2)
#         print("ff" + str(epoch))
#         model.train()

#         running_loss = 0.
#         correct = 0
#         total = 0

#         for images, labels in train_loader:
#             print(1)
#             images = images.to(device)
#             print("f" + str(len(images)))
#             labels = labels.to(device)

#             output = model(images)
#             loss = criterion(output, labels)

#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item() * images.size(0)
#             _, preds = torch.max(output, dim=1)
#             correct += (preds == labels).sum().item()
#             total += labels.size(0)

#         epoch_loss = running_loss / total
#         epoch_acc = correct / total
#         print(f"Epoch {epoch+1}/{epochs} "
#             f"train loss: {epoch_loss:.4f} "
#             f"train acc: {epoch_acc:.4f}")
    
#     model.eval()
#     correct = 0
#     total = 0

#     for images, labels in test_loader:
#         images = images.to(device)
#         labels = labels.to(device)

#         output = model(images)
#         _, preds = torch.max(output, dim=1)
#         correct += (preds == labels).sum().item()
#         total += labels.size(0)

#     test_acc = correct / total
#     print(f"Test accuracy: {test_acc:.4f}")

#     # del data
#     del model
#     gc.collect()
#     torch.cuda.empty_cache()

script_classification_cnn
2
ff0
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1
f1
1


KeyboardInterrupt: 